# 精确定位：

https://github.com/ApolloAuto/apollo/tree/master/modules/localization

最常用的方法是将汽车传感器所看到的内容与地图上显示的内容进行比较，车辆传感器可以测量车辆与静态障碍物之间的距离，如树木、电线杆、路标和墙壁等；

为了进行比对，必须进行坐标系转化；


## GNSS（全球导航系统）：

RTK

GPS是应用最广泛的GNSS；

GPS原理，由三部分组成：

![](./10location1.png)

卫星，在任何特定时间，大约有30颗GPS卫星在外层空间运行，各自距离地球表面约2万公里；

控制站，用于监视和控制卫星，其主要目的是让系统保持运行，并验证GPS广播信号的精确度；

GPS接收器，存在于手机，电脑，其撤等，在周围没有障碍物，天气良好的情况下，GPS接收器每次应至少检测到四颗GPS卫星；


GPS接收器实际上并不直接探测你和卫星之间的距离，首先测量信号的飞行时间，由于光速很大，及时少量时间误差也会引起巨大误差；

因此每颗卫星都配备高精度的院子钟；

为进一步减少误差，可以使用实时运动定位RTK，地面的多个基站，都知道自己精确的位置，每个基站通过GPS测量自己的位置，与已知的精确位置之间的误差，为GPS测量结果的误差，然后将这个误差发送给GPS接收器；以供其调整自身位置计算；

![](./10location2.png)

在RTK帮助下，GPS可以将误差限制在10cm以内，但是仍存在高楼和其他障碍物可能阻挡GPS信号的问题，使定位变得困难，甚至失败；

GPS另一个缺点是，更新频率太低大约10hz；

## 惯性导航

用三轴加速计的传感器测量加速度；


加速度计根据车辆坐标系记录测量的结果，陀螺仪传感器测量车辆在坐标系的位置

加速度计和陀螺仪是惯性测量单元IMU的主要组件；IMU以高频率更新，可达1000HZ，缺点是运动误差会累加；

我们只能用IMU在很短的时间范围内进行定位；

我们可以结合GPS和IMU来定位汽车，一方面IMU弥补了GPS更新频率较低的缺陷，另一方面GPS纠正了IMU的运动误差；
但即使GPS和IMU相结合，也不能完全解决定位问题；在峡谷，隧道，没有GPS信号更新；

## LiDAR定位

利用激光雷达，我们可以通过点云匹配来对汽车进行定位，该方法将来自激光雷达传感器的检测数据，
与预先存在的高精度地图连续匹配，通过这种比较可获知汽车在高精度地图上的全球位置和行驶方向；

![](./10location3.png)

有许多算法可用于匹配点云，迭代最近点(ICP)是一种方法，假设我们想对两次点云扫描进行匹配，对于第一次
扫描中的每个点，我们需要找到了一次扫描中最近接的匹配点，最终我们会收集到许多匹配点对，把每对点之间的距离误差相加；
然后计算平均距离误差，我们的目标是通过点云旋转，和平移来最大限度地降低这一平均距离误差。

![](./10location4.png)

一旦我们最大限度地降低了点云之间的误差，就可以在传感器扫描和地图之间找到匹配，将通过传感器扫描到的车辆位置
转换为全球地图上的位置，并计算出在地图上的精确位置；

![](./10location5.png)

滤波算法是另一种LiDAR定位方法, 滤波算法可消除冗余信息，并在地图上找到最可能的车辆位置，

Apollo使用了直方图滤波算法，也叫误差平方和算法(SSD),
将通过传感器扫描的点云滑过地图上的每个位置，在每个位置计算扫描点和高精度地图上的对应点之间的误差或距离，然后对误差的平方求和；

和越小，扫描结果与题图匹配的越好；

![](./10location6.png)


卡尔曼滤波是另一种LiDAR定位方法，该算法根据过去的状态和新的传感器测量结果预测我们当前的状态；

卡尔曼滤波使用了预测更新周期，根据之前的状态以及移动距离和方向的估计，来估计新的位置；

使用传感器测量的位置来纠正估计；

使用概率规则，将不完美的传感器测量结果与我们现有的位置预测结合起来，永远遵循这个预测更新周期；

LiDAR定位的主要优势在于稳健性，只要从高精度地图开始，并且存在有效的传感器，我们就始终可以进行定位；

主要缺点，在于难以构建高精度地图，并使其保持最新，事实上几乎不可能保持最新，因为几乎每个地图均包含瞬态元素(transient elements),汽车，行人路上垃圾；

这些元素都在不断发生变化，

**直方图**：https://classroom.udacity.com/courses/cs373/lessons/48739381/concepts/487350240923

**卡尔曼滤波**：https://classroom.udacity.com/courses/cs373/lessons/48723604/concepts/486709880923

## 视觉定位

图像是最容易收集的信息，摄像头种类繁多且便宜，通过图像时间精确定位非常困难，实际上，摄像头图像通常与来自

其他传感器的数据相结合，以准确定位车辆，将摄像头数据与地图,GPS相结合，效果比单独使用摄像头更好；

常用的滤波算法，粒子滤波：https://classroom.udacity.com/courses/cs373/lessons/48704330/concepts/487500080923

    
图像数据很容易获得，缺点是缺乏三维信息和对三维地图的依赖；





## Apollo定位

 Apollo使用基于GPS,IMU和激光雷达的多传感器融合定位系统，利用了不同传感器的互补优势；
 
 提高了稳定性和准确性。 Apollo定位模块依赖于IMU,GPS,激光雷达和高精度地图，这些传感器支持GNSS定位和LiDAR定位；
 
 GNSS输出位置和速度，LiDAR输出位置和行进方向信息

    
![](./10location7.png)

融合框架通过卡尔曼滤波将这些输出结合在一起，

在Apollo中惯性导航解决方案，用于卡尔曼滤波的预测步骤，GNSS和LiDAR定位用于卡尔曼滤波的测量结果更新步骤；




关于 Apollo 定位的论文：https://arxiv.org/pdf/1711.05805.pdf
    
卡尔曼滤波：https://classroom.udacity.com/courses/cs373/lessons/48723604/concepts/486709880923

Apollo关于定位资料：https://github.com/ApolloAuto/apollo/tree/master/modules/localization